[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/insactor-colab/blob/main/insactor_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/insactor
%cd /content/insactor

!pip install -q https://github.com/camenduru/wheels/releases/download/colab/mmcv_full-1.7.1-cp310-cp310-linux_x86_64.whl
!pip install -q streamlit brax==0.0.15 chex==0.1.5 optax==0.1.3 flax==0.6.1 -U
!pip install -q git+https://github.com/openai/CLIP
# !pip install -q -U "jax==0.4.2" "jaxlib==0.4.2" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -U jax==0.4.2 jaxlib==0.4.2 -U

!mkdir pretrained_models 
!wget https://huggingface.co/camenduru/insactor/resolve/main/planner_humanml.pth -O /content/insactor/pretrained_models/planner_humanml.pth
!wget https://huggingface.co/camenduru/insactor/resolve/main/skill_human_0.001.pkl -O /content/insactor/pretrained_models/skill_human_0.001.pkl

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

id_rsa_file = "/content/id_rsa"
id_rsa_pub_file = "/content/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

gen_key(id_rsa_file)

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:7860 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/insactor
!streamlit run demo.py --server.port 7860